<a href="https://colab.research.google.com/github/waundme/IntelligentVehicles/blob/main/Intelligent_Vehicles_und_Grobid_im_Batch_Mode%2C_XML_TEI_und_ElasticSearch%2C_einfachstes_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sci-Hub

In [ ]:
# Sci-Hub-DOIs herunterladen und entpacken.
!wget https://www.sci-hub.st/datasets/dois-2022-02-12.7z
!7za x dois-2022-02-12.7z

In [2]:
# Alle Intelligent-Vehicle-DOIs rausgreppen und in dois.txt speichern.
!grep -a 10.1109/tiv /content/sci-hub-doi-2022-02-12.txt > dois.txt

# scitopdf

In [ ]:
# scitopdf installieren
!git clone https://github.com/dougy147/scitopdf
!cd scitopdf && make install

In [ ]:
# Alle Intelligent-Vehicle-PDFs herunterladen und in /content/input speichern. 
# Bei Paper 322 scheint es einen Fehler zu geben und Script hängt, läuft aber schließlich durch.
# Möglicherweise Parameter -q (quiet) hinzufügen, damit der Output-Buffer hier nicht volläuft.
#!scitopdf -q -D /content/input -l dois.txt
!scitopdf -D /content/input -l dois.txt

# GrobId

In [ ]:
# GrobId herunterladen und entpacken.
!wget https://github.com/kermitt2/grobid/archive/0.7.2.zip
!unzip 0.7.2.zip

In [ ]:
# GrobId bauen und installieren. Das dauert 4 Minuten oder so, weil überkompliziertes Java-Build-System.
!cd /content/grobid-0.7.2 && ./gradlew clean install

In [7]:
# Output-Directory anlegen.
import os
os.mkdir("/content/output")

In [ ]:
# GrobId im Batch-Modus. Nimmt alle PDFs im Input-Verzeichnis und konvertiert sie nach XML/TEI im Output-Verzeichnis.
# Gibt ein paar Fehler, ich glaube, weil einige PDFs leer sind, keinen Inhalt haben.
# Braucht eine halbe Stunde oder so.
# -Xmx4G allokiert der JVM 4 GB RAM, möglicherweise muß man das höher setzen, Log/Output beobachten.
!java -Xmx4G -jar /content/grobid-0.7.2/grobid-core/build/libs/grobid-core-0.7.2-onejar.jar -gH /content/grobid-0.7.2/grobid-home -dIn /content/input/ -dOut /content/output -exe processFullText 

# TEI/XML

In [ ]:
# GrobId-TEI-XML-Parser installieren
# Siehe: https://pypi.org/project/grobid-tei-xml/
!pip install grobid_tei_xml

In [10]:
# Paper-Daten mit GrobId-TEI-XML-Parser auslesen:
# So könnte man auch eine "Rename"-Funktion implementieren: Autoren und Titel stehen ja jetzt im XML/TEI.
import grobid_tei_xml

# Für jede Datei im Output-Folder:
for file in os.listdir("/content/output"):
    # Nur XML-Dateien
    if file.endswith(".xml"):
        # Parsen und Metadaten auslesen:
        with open(f"/content/output/{file}", 'r') as xml_file:
          doc = grobid_tei_xml.parse_document_xml(xml_file.read())
          print("file name: " + file)
          # Titel
          if doc.header.title is not None:
            print("title: " + doc.header.title)
          # Autoren
          if doc.header.authors is not None:
            print("authors: " + ", ".join([a.full_name for a in doc.header.authors]))
          # DOI
          if doc.header.doi is not None:
            print("doi: " + str(doc.header.doi))
          # Abstract
          if doc.abstract is not None:
            print("abstract: " + doc.abstract)
          print("")
          # Der Text des Papers wäre in doc.body, den würde man in irgendeinem NLP-Tool weiterverarbeiten.
          #if doc.body is not None:
            #print(doc.body)
          # Fußnoten
          if doc.citations is not None:
            for citation in doc.citations:
              print("citation authors: " + ", ".join([author.full_name for author in citation.authors]))

# "Rename"-Script

In [ ]:
# "Rename"
import os
import shutil

for file in os.listdir("/content/output"):
  # Nur XML-Dateien
  if file.endswith(".xml"):
      # Parsen und Titel auslesen:
      with open(f"/content/output/{file}", 'r') as xml_file:
        doc = grobid_tei_xml.parse_document_xml(xml_file.read())

        # Titel
        if doc.header.title is not None:
        # Titel bei 250 Zeichen abschneiden, weil sonst zu lange Dateinamen für das Dateisystem. (Linux MAX_FILEPATH = 255)
          title = (doc.header.title[:250]) if len(doc.header.title) > 250 else doc.header.title
          # Ungültige Dateinamen-Buchstaben löschen. (Da sind zum Teil komische Sonderzeichen in den Titeln, die Linux nicht will.)
          title = "".join(x for x in title if x.isprintable())
        # Wie ist der Pfad zum originalen PDF?
        pdf_path = "/content/input/" + file.replace(".tei.xml", ".pdf")
        
        # Umbenennen
        shutil.move(pdf_path, f"/content/input/{title}.pdf")

# spaCy-NLP

In [ ]:
# NLP-Library und englisches Language Model installieren.
# spaCy bietet übrigens GPU-Unterstützung, nutzen wir jetzt aber nicht.
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
# Beispiel für die Weiterverarbeitung in einer NLP-Library, hier spaCy.
# Siehe https://spacy.io/
# Wortvektoren, Entity Linking, whatever.
# Da kommt noch ziemlich viel Mist raus, aber man würde spacy zum Postprocessing benutzen.

import spacy

# Kleines englisches Language Model laden
nlp = spacy.load("en_core_web_sm")

# Für jede Datei im Output-Folder:
for file in os.listdir("/content/output"):
    # Nur XML-Dateien
    if file.endswith(".xml"):
        with open(f"/content/output/{file}", 'r') as xml_file:
          doc = grobid_tei_xml.parse_document_xml(xml_file.read())
          print("file name: " + file)
          if doc.body is not None:
            doc = nlp(doc.body)

            # Alle Sätze einzeln ausgeben:
            for sent in doc.sents:
              print(sent)
            # Entity Recognition
            for entity in doc.ents:
              print(entity.text, entity.label_)

# Volltextsuche: ElasticSearch siehe https://www.elastic.co/elasticsearch/

In [ ]:
# ElasticSearch Client-Library installieren
%%capture

!pip install elasticsearch==7.14.0

In [ ]:
# Library-Importe
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm 
  import datetime
  from elasticsearch import helpers
  
except Exception as e:
  print(f"error: {e}")

In [ ]:
# Elasticsearch 7.0.0 downloaden und entpacken
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Daemon-Instanz von Elasticsearch erstellen.
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # als Daemon
                 )

In [ ]:
# Dieser Teil ist wichtig: warten. ElasticSearch braucht eine Weile, bis es hochgefahren ist.
import time
time.sleep(20)

In [ ]:
%%bash
# Wenn Du hier 1 root und 2 daemon Prozesse siehst, ist Elasticsearch erfolgreich gestartet.
ps -ef | grep elasticsearch

In [ ]:
# Antwortet ElasticSearch?
!curl -sX GET "localhost:9200/"

In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Gucken, ob Python den ElasticSearch-Server erreichen kann.
es.ping()

In [ ]:
# Dataset erstellen, rudimentär, ist nur ein Showcase.
dataset = pd.DataFrame({"name": [], "body": []})

for file in os.listdir("/content/output"):
  # Nur XML-Dateien
  if file.endswith(".xml"):
      # Parsen und Body auslesen
      with open(f"/content/output/{file}", 'r') as xml_file:
        doc = grobid_tei_xml.parse_document_xml(xml_file.read())

        # Name der Datei ("name") und Text des Papers ("body") in eine Tabellenform bringen
        if doc.body is not None:
          row = {"name": file, "body": doc.body}
          dataset = dataset.append(row, ignore_index=True)

print(f"shape of dataset: {dataset.shape}")
# Vorschau:
dataset.head()

In [ ]:
# Einstellungen ("settings") und Schema ("mappings") für den Elasticsearch-Index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "name":{
                "type":"text"
            },
            "body":{
                "type":"text"
            }
        }
    }
}

In [ ]:
# Generische Funktion, erstellt JSON-formatierte Dictionaries für Elasticsearch.
def json_formatter(dataset, index_name, index_type='_doc'):
    try:
        List = []
        columns = dataset.columns
        for idx, row in dataset.iterrows():
            dic = {}
            dic['_index'] = index_name
            dic['_type'] = index_type
            source = {}
            for i in dataset.columns:
                source[i] = row[i]
            dic['_source'] = source
            List.append(dic)
        return List
    
    except Exception as e:
        print("There is a problem: {}".format(e))

In [ ]:
# Erstellt den Index "paper_index" in ElasticSearch mit den Einstellungen, die wir oben definiert haben.
PAPER_INDEX = es.indices.create(index="paper_index", ignore=[400,404], body=Settings)
PAPER_INDEX

In [ ]:
# JSON-formatiertes Dataset für ElasticSearch
json_Formatted_dataset = json_formatter(dataset=dataset, index_name='paper_index', index_type='_doc')
# Vorschau:
json_Formatted_dataset[0]

In [ ]:
# Um das Dataset in ElasticSearch zu importieren, nutzen wir das ElasticSearch Bulk API.
try:
    res = helpers.bulk(es, json_Formatted_dataset)
    print("Successfully imported to ElasticSearch.")
except Exception as e:
    print(f"error: {e}")

In [ ]:
# Simple Suche nach "LIDAR"
query = es.search(
    index="paper_index",
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"body":"LIDAR"}}
                ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output